### Making the County Table
This is the table that has the county name, county id, and corresponding state id.
It will be used to pull state-level information into our examples for each county

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import json

In [2]:
# Load all state ids from table
df_state_table = pd.read_csv('state_table.csv')
print(df_state_table.head())
df_county_table = pd.DataFrame()

      id        name
0  27282     Alabama
1  27283      Alaska
2  27284     Arizona
3    149    Arkansas
4  27286  California


In [ ]:
# Loop over all state ids to get the list of counties within each state
states_wo_counties = []
no_categories = []

states_list = list(df_state_table['id'].values)
# Alaska, Louisiana, and Virgin Islands dont have counties so we will deal with those later
states_list.remove(27283)
states_list.remove(27300)
states_list.remove(33605)

for state_id in states_list:
    time.sleep(2)
    try:        
        params = {
            'category_id': state_id,
            'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
            'file_type': 'json'
        }
        r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
        res = r.json()
    except:
        print('Error')
        print(state_id)
    try:
        found_counties = False
        if 'categories' in list(res.keys()):
            for child in res['categories']:
                if child['name'] == 'Counties':
                    state_counties_id = child['id']
                    found_counties = True
        else:
            no_categories.append(state_id)


        if found_counties:
            params = {
                'category_id': state_counties_id,
                'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
                'file_type': 'json'
            }
            r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
            res = r.json()
            temp = pd.DataFrame(res['categories'])
            temp['state_id'] = state_id
            df_county_table = df_county_table.append(temp)
        else:
            states_wo_counties.append(state_id)
    except:
        print('Error')
        print(state_id)
        print(res)

In [ ]:
# Print results
df_county_table.reset_index(inplace=True, drop=True)
print(df_county_table.shape)
print(states_wo_counties)
print(no_categories)

In [ ]:
# Alaska
params = {
    'category_id': 27283,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

for child in res['categories']:
    if child['name'] == 'Census Areas and Boroughs':
        state_counties_id = child['id']

params = {
    'category_id': state_counties_id,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

temp = pd.DataFrame(res['categories'])
temp['state_id'] = 27283
df_county_table = df_county_table.append(temp)

In [ ]:
# Louisiana
params = {
    'category_id': 27300,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

for child in res['categories']:
    if child['name'] == 'Parishes':
        state_counties_id = child['id']

params = {
    'category_id': state_counties_id,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

temp = pd.DataFrame(res['categories'])
temp['state_id'] = 27300
df_county_table = df_county_table.append(temp)

In [7]:
# Virgin Islands
df_county_table.append({'id': 33605, 'name': 'Virgin Islands', 'state_id': 33605, 'parent_id': np.NaN}, ignore_index=True)

,id,name,parent_id,state_id
0,27336.0,"Autauga County, AL",27335.0,27282
1,27337.0,"Baldwin County, AL",27335.0,27282
2,27338.0,"Barbour County, AL",27335.0,27282
3,27339.0,"Bibb County, AL",27335.0,27282
4,27340.0,"Blount County, AL",27335.0,27282
...,...,...,...,...
3172,28522.0,"West Baton Rouge Parish, LA",28461.0,27300
3173,28523.0,"West Carroll Parish, LA",28461.0,27300
3174,28524.0,"West Feliciana Parish, LA",28461.0,27300
3175,28525.0,"Winn Parish, LA",28461.0,27300


In [8]:
print(df_county_table.shape)

(3176, 4)


In [9]:
df_county_table.rename(columns={'id':'county_id'}, inplace=True)

KeyError: "['parent_id'] not found in axis"

In [11]:
df_county_table.drop(labels=['parent_id'], axis=1, inplace=True)

KeyError: "['parent_id'] not found in axis"

In [12]:
df_county_table.to_csv('county_table.csv', index=False)